# Back Propagation 이용

In [17]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
import time
import numpy as np
from collections import OrderedDict as OD

from dataset.mnist import load_mnist as load
import matplotlib.pyplot as plt
import pickle

In [48]:
class Affine: # 정리된 계층(활성함수 like ReLU에 넣기 전)
    def __init__(self, W, b):
        self.W = W # 행렬값
        self.b = b # 역치
        self.x = None # 입력값
        self.dW = None # W변화량(기울기)
        self.db = None # b변화량(기울기)
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b # 값 계산
        
        return out
    
    def backward(self, dout): # 역전파 첫 입력값
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

In [49]:
class Softmax:
    def __init__(self):
        self.loss = None
        self.y = None
        self.x = None
        
    def forward(self, x, t): # 입력과 정답에 대해
        self.t = t
        self.y = softmax(x) # softmax는 commom/function에 있음
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t)/batch_size # 어파인 원소 하나당 오차를 전파해야 함
        
        return dx

In [50]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [51]:
class Learning:
    def __init__(self, insize, hidsize, outsize, weight = 0.01):
        self.params = {}
        self.params['W1'] = weight * np.random.randn(insize, hidsize)
        self.params['W2'] = weight * np.random.randn(hidsize, outsize)
        self.params['b1'] = weight * np.zeros(hidsize)
        self.params['b2'] = weight * np.zeros(outsize)
        
        self.layers = OD()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = Softmax()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x) # 순전파 계산
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y,t) # 교차엔트로피 오차계산
    
    def grad_back(self, x, t):
        self.loss(x,t)
        
        dout = 1 # initialize
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['W2'] = self.layers['Affine2'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [85]:
(x_tr, t_tr), (x_te, t_te) = load(one_hot_label = True)

In [164]:
iters_num = 60000
train_size = x_tr.shape[0]
batch_size = 10000
learning_rate = 0.1

In [165]:
net = Learning(insize = 784, hidsize = 50, outsize = 10)

In [ ]:
now = time.time()
train_loss = []
for i in range(iters_num):
    slicing = np.random.choice(train_size, batch_size)
    x_batch = x_tr[slicing]
    t_batch = t_tr[slicing]
    
    gradient = net.grad_back(x_batch, t_batch)
    
    for key in ['W1', 'W2', 'b1', 'b2']:
        net.params[key] -= learning_rate * gradient[key]
        
    loss = net.loss(x_batch, t_batch)
    train_loss.append(loss)
    
    elapsed_time = (time.time()-now)
    second = elapsed_time%60
    minute = elapsed_time//60%60
    hour = elapsed_time//3600
    
    print('calc {0:.2f}% done. elapsed : {1:.0f}h {2:.0f}m {3:.0f}s'.format((i+1)*100/iters_num,hour, minute, second), end = '\r')


In [ ]:
with open('common/params_dict4.pkl', 'wb') as f:
    pickle.dump(net.params, f)
    

plt.plot(range(iters_num), train_loss)
plt.show()

학습법으로는 adam, 초기값으로는 He를 사용하는 것이 Relu에선 유리하다.

batch normalization

그냥 미분으로 3시간 걸리던 계산이 1초만에 뚝딱 됨. 개신기

In [ ]:
accuracy = 0
for i in range(60000):
    if np.argmax(softmax(net.predict(x_tr[i]))) == np.argmax(t_tr[i]):
        accuracy += 1
        
print(accuracy/60000)
        

In [ ]:
0.838 0.924 0.93(20000) 0.934(30000), #40000부턴 떨어짐